Create the dataset

In [6]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS fraud_dataset
OPTIONS(
location="us",
default_table_expiration_days=14 );

Query is running:   0%|          |

""


Create the table from the fraud data CSV.*italicized text*

In [7]:
%%bigquery
CREATE OR REPLACE EXTERNAL TABLE `qwiklabs-gcp-01-a7e68aa7e5b0.fraud_dataset.fraud_data`
OPTIONS (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/fraud_data_raw.csv'],
  skip_leading_rows = 1,
  allow_jagged_rows = false,
  allow_quoted_newlines = false,
  field_delimiter = ',',
  max_bad_records = 10
)


Query is running:   0%|          |

""


Modify the data in the original table and put it into a new table called fraud_training_data.

*   **One-hot encode the Employment_Status and Device_Type fields**
*   Break the age field into bins (e.g., 18–24, 25–34, 35–44, etc.)
*   Create a field called Income-to-Amount-Requested which is the ratio of those fields
*   Create a calculated field called Time_Since_Previous_Assistance
*   Change True/False fields to 0s and 1s

In [12]:
%%bigquery
CREATE OR REPLACE TABLE `qwiklabs-gcp-01-a7e68aa7e5b0.fraud_dataset.fraud_training_data` AS (
  SELECT
    -- Select all original columns, excluding those we are transforming to avoid duplication
    * EXCEPT (
      Employment_Status, Device_Type, Age, Previous_Assistance_Received, Supporting_Doc_Verified
    ),

    -- Step 1: One-hot encode Employment_Status
    -- Creates a new column for each category with a 1 or 0
    CASE WHEN Employment_Status = 'Employed' THEN 1 ELSE 0 END AS emp_status_employed,
    CASE WHEN Employment_Status = 'Unemployed' THEN 1 ELSE 0 END AS emp_status_unemployed,
    CASE WHEN Employment_Status = 'Self-Employed' THEN 1 ELSE 0 END AS emp_status_self_employed,

    -- Step 1: One-hot encode Device_Type
    CASE WHEN Device_Type = 'Mobile' THEN 1 ELSE 0 END AS device_type_mobile,
    CASE WHEN Device_Type = 'Desktop' THEN 1 ELSE 0 END AS device_type_desktop,
    CASE WHEN Device_Type = 'Tablet' THEN 1 ELSE 0 END AS device_type_tablet,

    -- Step 2: Break the age field into bins
    -- Creates a new categorical feature from the numerical age
    CASE
      WHEN Age BETWEEN 18 AND 24 THEN '18-24'
      WHEN Age BETWEEN 25 AND 34 THEN '25-34'
      WHEN Age BETWEEN 35 AND 44 THEN '35-44'
      WHEN Age BETWEEN 45 AND 54 THEN '45-54'
      WHEN Age BETWEEN 55 AND 64 THEN '55-64'
      ELSE '65+'
    END AS age_bin,

    -- Step 3: Create a ratio of Income to Amount Requested
    -- SAFE_DIVIDE prevents errors if Amount_Requested is 0
    SAFE_DIVIDE(Income, Amount_Requested) AS Income_to_Amount_Requested,

    -- Step 4: Create a field for Days Since Previous Assistance
    -- Calculates the number of days between the current date and the Previous_Assistance_Date.
    -- Returns NULL if Previous_Assistance_Date is NULL.
    DATE_DIFF(CURRENT_DATE(), Previous_Assistance_Date, DAY) AS Time_Since_Previous_Assistance,

    -- Step 5: Change boolean (True/False) fields to integers (0/1)
    -- Assumes you have boolean fields named Has_Fraud_History and Is_Repeat_Customer
    CAST(Previous_Assistance_Received AS INT64) AS Previous_Assistance_Received_int,
    CAST(Supporting_Doc_Verified AS INT64) AS Supporting_Doc_Verified_int

  FROM
    `qwiklabs-gcp-01-a7e68aa7e5b0.fraud_dataset.fraud_data`
);


Query is running:   0%|          |

""
